In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
train = pd.read_csv('/kaggle/input/titanic/train.csv')
test = pd.read_csv('/kaggle/input/titanic/test.csv')

In [4]:
train.head()

In [5]:
train.isnull().sum()

In [6]:
sns.heatmap(train.isnull())

In [7]:
train.describe()

In [8]:
train.describe(include = ['O'])

In [9]:
train.corr()

In [10]:
sns.heatmap(train.corr())

In [11]:
sns.set_style('whitegrid')
sns.countplot(x='Survived',data=train,hue = 'Sex')

In [12]:
train[['Sex','Survived']].groupby(['Sex']).mean().sort_values(by = 'Survived',ascending = False)

In [13]:
train[['Pclass','Survived']].groupby(['Pclass']).mean().sort_values(by = 'Survived',ascending = False)

In [14]:
train[['SibSp','Survived']].groupby(['SibSp']).mean().sort_values(by = 'Survived',ascending = False)

In [15]:
train[['Parch','Survived']].groupby(['Parch']).mean().sort_values(by = 'Survived',ascending = False)

In [16]:
train[['Embarked','Survived']].groupby(['Embarked']).mean().sort_values(by = 'Survived',ascending = False)

In [17]:
sns.displot(x= train['Age'].dropna(),kde=True)

In [18]:
sns.histplot(x='Age' ,hue = 'Survived', data = train)

In [19]:
sns.countplot(x='Pclass',data = train)

In [20]:
sns.countplot(x='Pclass',data = train,hue = 'Survived')

In [21]:
sns.pointplot(x='Embarked',data = train, y='Survived')

In [22]:
sns.countplot(x='Embarked',data = train,hue = 'Pclass')

In [25]:
grid = sns.FacetGrid(row='Embarked',col='Survived',hue = 'Sex',data = train)
grid.map(sns.histplot, 'Pclass')
grid.add_legend()

In [26]:
sns.pointplot(x='SibSp',data = train, y='Survived')

In [31]:
pd.crosstab(train.SibSp,train.Pclass).style.background_gradient()

In [28]:
sns.pointplot(x='Parch',data = train, y='Survived')

In [53]:
sns.displot(data= train, x='Fare',bins=15,row='Pclass',hue='Survived')